# Runs brainvisa preprocessing

This notebook creates the whole brainvisa preprocessing pipeline to feed to deep learning algorithms.
Note that you need brainvisa installed or to run the server using the brainvisa singularity.

# Imports

General imports

In [1]:
import sys
import os
from os.path import join
import glob
import json

Deep_folding imports

In [2]:
from deep_folding.anatomist_tools import transform
from deep_folding.anatomist_tools import bounding_box
from deep_folding.anatomist_tools import dataset_gen_pipe
print((os.path.dirname(transform.__file__)))
print((os.path.dirname(bounding_box.__file__)))
print((os.path.dirname(dataset_gen_pipe.__file__)))

/host/home/jc225751/Program/deep_folding/deep_folding/anatomist_tools
/host/home/jc225751/Program/deep_folding/deep_folding/anatomist_tools
/host/home/jc225751/Program/deep_folding/deep_folding/anatomist_tools


Constants

In [3]:
_ALL_SUBJECTS = -1

# Creates useful functions

In [4]:
def check_directory(directory_path):
    """Checks directory path and returns absolute path"""
    directory_path = os.path.abspath(directory_path)
    if os.path.isdir(directory_path):
        print((directory_path + ' is a directory'))
    else:
        print((directory_path + ' does not exist or is not a directory.'))
    return directory_path

# Variables used by all sub-computations

The following boolean variables decide which pprocessing to run:

In [5]:
run_transform = False  # If set to True, it generates new transform files
run_bbox = False  # If set to True, it generates new bounding boxes
run_crop = True  # If set to True, it generates crops

We now assign path names and other user-specific variables.

In [6]:
path_to_root = '/neurospin'  # Could be either /neuropsin, or /nfs/neurospin, for example

The unsupervised source directory is where the unsupervised database lies. It contains the morphologist analysis subfolder ANALYSIS/3T_morphologist


In [7]:
unsupervised_src_dir = check_directory(join(path_to_root, 'hcp'))

/neurospin/hcp does not exist or is not a directory.


The supervised source directories are where lies the database that has been manually labelled. It is a list of full pathes towards the manually labelled datasets.

In [8]:
human_supervised_dir = join(path_to_root, 'dico/data/bv_databases/human')
supervised_src_dir = [check_directory(join(human_supervised_dir, 'pclean/all'))
                     ]
path_to_graph = ["t1mri/t1/default_analysis/folds/3.3/base2018_manual"
                 ]

/neurospin/dico/data/bv_databases/human/pclean/all does not exist or is not a directory.


# Generates transform files

### User variables

The transform directory is where the transform files will be saved. It will be created afterwards if it does not exist

In [9]:
transform_dir = check_directory(join(path_to_root, 'dico/deep_folding_data/data', 'transform'))

/neurospin/dico/deep_folding_data/data/transform does not exist or is not a directory.


### Generates transform files (actual program)

In [10]:
args = "--help"
argv = args.split(' ')
transform.main(argv)

usage: transform.py [-h] [-s SRC_DIR] [-t TGT_DIR] [-n NB_SUBJECTS]

Generates transformation files

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR, --src_dir SRC_DIR
                        Source directory where the MRI data lies. Default is :
                        /neurospin/hcp
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the output
                        transformation files. Default is :
                        /neurospin/dico/deep_folding_data/test/transform
  -n NB_SUBJECTS, --nb_subjects NB_SUBJECTS
                        Number of subjects to take into account, or 'all'.0
                        subject is allowed, for debug purpose.Default is : all


In [11]:
if run_transform:
    transform.transform_to_spm(src_dir=unsupervised_src_dir, tgt_dir=transform_dir, number_subjects=_ALL_SUBJECTS)

### Result analysis

Prints the number of transform files as well as the content of the json file

In [12]:
transform_files = glob.glob(transform_dir + '/*.trm')
print("Number of transform files : ", len(transform_files))

Number of transform files :  0


In [13]:
transform_json_file = transform_dir + "/transform.json"
print(open(transform_json_file, "r").read())

FileNotFoundError: [Errno 2] No such file or directory: '/neurospin/dico/deep_folding_data/data/transform/transform.json'

# Generates bounding boxes

### User variables

In [14]:
bbox_dir = check_directory(join(path_to_root, 'dico/deep_folding_data/data', 'bbox'))

/neurospin/dico/deep_folding_data/data/bbox does not exist or is not a directory.


Lists the sulci of the left side that we want to analyze:

In [15]:
sulci_left = ['S.T.s.ter.asc.ant._left', 'S.T.s.ter.asc.post._left']

Lists the sulci of the right side that we want to analyze:

In [16]:
sulci_right = ['S.T.s.ter.asc.ant._right', 'S.T.s.ter.asc.post._right']

In [17]:
image_normalized_spm = join(unsupervised_src_dir, 
                            'ANALYSIS/3T_morphologist/100206',
                            't1mri/default_acquisition',
                            'normalized_SPM_100206.nii')

### Generates bounding boxes (actual program)

In [18]:
args = "--help"
argv = args.split(' ')
bounding_box.main(argv)

usage: bounding_box.py [-h] [-s SRC_DIR [SRC_DIR ...]] [-t TGT_DIR]
                       [-u SULCUS] [-i SIDE] [-m IMAGE_NORMALIZED_SPM]
                       [-p PATH_TO_GRAPH] [-n NB_SUBJECTS]

Computes bounding box around the named sulcus

optional arguments:
  -h, --help            show this help message and exit
  -s SRC_DIR [SRC_DIR ...], --src_dir SRC_DIR [SRC_DIR ...]
                        Source directory where the MRI data lies. If there are
                        several directories, add all directories one after the
                        other. Example: -s DIR_1 DIR_2. Default is :
                        /neurospin/lnao/PClean/database_learnclean/all/
  -t TGT_DIR, --tgt_dir TGT_DIR
                        Target directory where to store the output
                        transformation files. Default is :
                        /neurospin/dico/deep_folding_data/test/bbox
  -u SULCUS, --sulcus SULCUS
                        Sulcus name around which we determine th

In [19]:
if run_bbox:
    for sulcus in sulci_left:
        bounding_box.bounding_box(src_dir=supervised_src_dir, 
                                  path_to_graph=path_to_graph,
                                  tgt_dir=bbox_dir,
                                  sulcus=sulcus, side='L',
                                  image_normalized_spm=image_normalized_spm,
                                  number_subjects=_ALL_SUBJECTS)
    for sulcus in sulci_right:
        bounding_box.bounding_box(src_dir=supervised_src_dir, 
                                  path_to_graph=path_to_graph,
                                  tgt_dir=bbox_dir,
                                  sulcus=sulcus, side='R',
                                  image_normalized_spm=image_normalized_spm,
                                  number_subjects=_ALL_SUBJECTS)

# Generates crops

### User variables 

In [20]:
interp = 'nearest'

In [21]:
crop_dir = check_directory(join(path_to_root, 'dico/deep_folding_data/data', 'crops/STS_branches/nearest/original'))

/neurospin/dico/deep_folding_data/data/crops/STS_branches/nearest/original does not exist or is not a directory.


### Generates crops (actual program)

In [22]:
if run_crop:
    # Runs on left hemisphere
    dataset_gen_pipe.dataset_gen_pipe(src_dir=unsupervised_src_dir,
                                      tgt_dir=crop_dir,
                                      transform_dir=transform_dir,
                                      bbox_dir=bbox_dir,
                                      list_sulci=sulci_left,
                                      side='L',
                                      interp=interp,
                                      number_subjects=_ALL_SUBJECTS)
    # Runs on right hemisphere
    dataset_gen_pipe.dataset_gen_pipe(src_dir=unsupervised_src_dir,
                                      tgt_dir=crop_dir,
                                      transform_dir=transform_dir,
                                      bbox_dir=bbox_dir,
                                      list_sulci=sulci_right,
                                      side='R',
                                      interp=interp,
                                      number_subjects=_ALL_SUBJECTS)
    

FileNotFoundError: [Errno 2] No such file or directory: '/neurospin/dico/deep_folding_data/data/bbox/L/S.T.s.ter.asc.ant._left.json'